In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

  Using cached mediapipe-0.9.0.1-cp39-cp39-win_amd64.whl (49.8 MB)
  Using cached opencv_python-4.7.0.68-cp37-abi3-win_amd64.whl (38.2 MB)
  Using cached opencv_contrib_python-4.7.0.68-cp37-abi3-win_amd64.whl (44.9 MB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached flatbuffers-23.1.4-py2.py3-none-any.whl (26 kB)


In [2]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv

In [3]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

# Live View

In [5]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Live Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# Capturing Landmarks & Exporting to CSV

In [7]:
import csv
import os
import numpy as np

In [8]:
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)
num_coords

501

In [23]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [24]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [28]:
class_name = "Heart Pain"

In [29]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow(str(class_name), image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# Training a custom model

In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [31]:
df = pd.read_csv('coords.csv')

In [32]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Normal,0.473864,0.209743,-0.913422,0.999301,0.499494,0.158025,-0.914669,0.998767,0.519605,...,-0.001206,0.0,0.534358,0.148146,0.002527,0.0,0.539287,0.142657,0.002047,0.0
1,Normal,0.478457,0.196497,-0.662465,0.999362,0.506243,0.148966,-0.649692,0.998874,0.525987,...,0.000672,0.0,0.538303,0.148784,0.004742,0.0,0.542901,0.143741,0.004462,0.0
2,Normal,0.478574,0.191807,-0.618810,0.999419,0.506701,0.146689,-0.611132,0.998974,0.526386,...,0.000407,0.0,0.540853,0.149374,0.004584,0.0,0.545477,0.144215,0.004329,0.0
3,Normal,0.479381,0.188323,-0.633386,0.999464,0.508491,0.144196,-0.623398,0.999055,0.527954,...,-0.000063,0.0,0.543700,0.148233,0.003907,0.0,0.548347,0.143096,0.003651,0.0
4,Normal,0.482769,0.186188,-0.618310,0.999507,0.512321,0.142115,-0.604685,0.999128,0.531552,...,0.000841,0.0,0.544131,0.146286,0.003864,0.0,0.548832,0.140816,0.003572,0.0


In [33]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
545,Heart Pain,0.548815,0.354086,-0.676022,0.999895,0.564110,0.290627,-0.686093,0.999798,0.578483,...,-0.026321,0.0,0.568015,0.302486,-0.037817,0.0,0.570701,0.294453,-0.039742,0.0
546,Heart Pain,0.548250,0.361953,-0.664893,0.999874,0.563758,0.298124,-0.677264,0.999756,0.578262,...,-0.026821,0.0,0.572152,0.309080,-0.036401,0.0,0.575333,0.302045,-0.038331,0.0
547,Heart Pain,0.557248,0.364671,-0.860014,0.999884,0.571190,0.303192,-0.845712,0.999776,0.585371,...,-0.032930,0.0,0.577392,0.297597,-0.040123,0.0,0.579780,0.288292,-0.041852,0.0
548,Heart Pain,0.556050,0.359006,-0.840282,0.999894,0.571072,0.298120,-0.820395,0.999795,0.585117,...,-0.028618,0.0,0.576563,0.296654,-0.036892,0.0,0.579695,0.289666,-0.038694,0.0
549,Heart Pain,0.542079,0.357262,-0.811777,0.999902,0.558801,0.296159,-0.794546,0.999811,0.573123,...,-0.028566,0.0,0.561146,0.278804,-0.040771,0.0,0.564527,0.272615,-0.043046,0.0


In [34]:
df[df['class']=='Normal']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Normal,0.473864,0.209743,-0.913422,0.999301,0.499494,0.158025,-0.914669,0.998767,0.519605,...,-0.001206,0.0,0.534358,0.148146,0.002527,0.0,0.539287,0.142657,0.002047,0.0
1,Normal,0.478457,0.196497,-0.662465,0.999362,0.506243,0.148966,-0.649692,0.998874,0.525987,...,0.000672,0.0,0.538303,0.148784,0.004742,0.0,0.542901,0.143741,0.004462,0.0
2,Normal,0.478574,0.191807,-0.618810,0.999419,0.506701,0.146689,-0.611132,0.998974,0.526386,...,0.000407,0.0,0.540853,0.149374,0.004584,0.0,0.545477,0.144215,0.004329,0.0
3,Normal,0.479381,0.188323,-0.633386,0.999464,0.508491,0.144196,-0.623398,0.999055,0.527954,...,-0.000063,0.0,0.543700,0.148233,0.003907,0.0,0.548347,0.143096,0.003651,0.0
4,Normal,0.482769,0.186188,-0.618310,0.999507,0.512321,0.142115,-0.604685,0.999128,0.531552,...,0.000841,0.0,0.544131,0.146286,0.003864,0.0,0.548832,0.140816,0.003572,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,Normal,0.517229,0.219295,-0.552138,0.999945,0.539767,0.177083,-0.525700,0.999897,0.555034,...,0.001447,0.0,0.566656,0.190217,0.006798,0.0,0.570644,0.185059,0.006894,0.0
172,Normal,0.517452,0.219899,-0.552390,0.999949,0.539882,0.177339,-0.524995,0.999906,0.555249,...,0.001228,0.0,0.566290,0.191066,0.006780,0.0,0.570299,0.186011,0.006832,0.0
173,Normal,0.517539,0.218828,-0.560990,0.999952,0.539973,0.176901,-0.530925,0.999912,0.555308,...,0.005304,0.0,0.560943,0.172207,0.010403,0.0,0.565041,0.167465,0.010483,0.0
174,Normal,0.515974,0.214607,-0.580061,0.999956,0.538878,0.174925,-0.547840,0.999917,0.554677,...,0.005993,0.0,0.559702,0.167575,0.011034,0.0,0.563807,0.161725,0.011278,0.0


In [35]:
df[df['class']=='Heart Pain']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
176,Heart Pain,0.516903,0.220702,-0.875240,0.999255,0.541797,0.176847,-0.852671,0.998708,0.557004,...,0.000591,0.0,0.566350,0.167280,0.005096,0.0,0.571113,0.160888,0.004924,0.0
177,Heart Pain,0.516623,0.219195,-0.580764,0.999285,0.540977,0.176790,-0.562531,0.998704,0.556425,...,0.001536,0.0,0.560083,0.169162,0.005497,0.0,0.564304,0.163643,0.005284,0.0
178,Heart Pain,0.516429,0.217973,-0.577591,0.999325,0.540702,0.176519,-0.559743,0.998735,0.556301,...,0.001082,0.0,0.559776,0.170310,0.004780,0.0,0.564146,0.164434,0.004637,0.0
179,Heart Pain,0.515978,0.215877,-0.529028,0.999387,0.540312,0.175407,-0.509195,0.998828,0.556079,...,0.000921,0.0,0.558104,0.170658,0.004687,0.0,0.562512,0.164871,0.004565,0.0
180,Heart Pain,0.515281,0.214343,-0.539364,0.999443,0.539676,0.174662,-0.516418,0.998914,0.555611,...,0.002148,0.0,0.558509,0.171389,0.004998,0.0,0.562876,0.165443,0.004894,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,Heart Pain,0.548815,0.354086,-0.676022,0.999895,0.564110,0.290627,-0.686093,0.999798,0.578483,...,-0.026321,0.0,0.568015,0.302486,-0.037817,0.0,0.570701,0.294453,-0.039742,0.0
546,Heart Pain,0.548250,0.361953,-0.664893,0.999874,0.563758,0.298124,-0.677264,0.999756,0.578262,...,-0.026821,0.0,0.572152,0.309080,-0.036401,0.0,0.575333,0.302045,-0.038331,0.0
547,Heart Pain,0.557248,0.364671,-0.860014,0.999884,0.571190,0.303192,-0.845712,0.999776,0.585371,...,-0.032930,0.0,0.577392,0.297597,-0.040123,0.0,0.579780,0.288292,-0.041852,0.0
548,Heart Pain,0.556050,0.359006,-0.840282,0.999894,0.571072,0.298120,-0.820395,0.999795,0.585117,...,-0.028618,0.0,0.576563,0.296654,-0.036892,0.0,0.579695,0.289666,-0.038694,0.0


In [36]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [38]:
y_test

471    Heart Pain
290    Heart Pain
537    Heart Pain
269    Heart Pain
124        Normal
          ...    
452    Heart Pain
454    Heart Pain
0          Normal
298    Heart Pain
508    Heart Pain
Name: class, Length: 165, dtype: object

## Classification Model

In [39]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [40]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [41]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [42]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [43]:
fit_models['rc'].predict(X_test)

array(['Heart Pain', 'Heart Pain', 'Heart Pain', 'Heart Pain', 'Normal',
       'Heart Pain', 'Heart Pain', 'Heart Pain', 'Heart Pain', 'Normal',
       'Heart Pain', 'Heart Pain', 'Normal', 'Heart Pain', 'Heart Pain',
       'Heart Pain', 'Heart Pain', 'Normal', 'Heart Pain', 'Heart Pain',
       'Heart Pain', 'Heart Pain', 'Normal', 'Normal', 'Heart Pain',
       'Heart Pain', 'Heart Pain', 'Normal', 'Normal', 'Heart Pain',
       'Normal', 'Heart Pain', 'Normal', 'Heart Pain', 'Normal',
       'Heart Pain', 'Normal', 'Heart Pain', 'Heart Pain', 'Normal',
       'Heart Pain', 'Normal', 'Heart Pain', 'Normal', 'Normal', 'Normal',
       'Heart Pain', 'Heart Pain', 'Heart Pain', 'Heart Pain', 'Normal',
       'Heart Pain', 'Normal', 'Heart Pain', 'Normal', 'Normal',
       'Heart Pain', 'Normal', 'Heart Pain', 'Heart Pain', 'Heart Pain',
       'Normal', 'Normal', 'Heart Pain', 'Heart Pain', 'Normal',
       'Heart Pain', 'Heart Pain', 'Heart Pain', 'Heart Pain', 'Normal',
       'Hear

## Dump model

In [44]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [45]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 0.9939393939393939
rc 1.0
rf 1.0
gb 0.9939393939393939


In [46]:
fit_models['rf'].predict(X_test)

array(['Heart Pain', 'Heart Pain', 'Heart Pain', 'Heart Pain', 'Normal',
       'Heart Pain', 'Heart Pain', 'Heart Pain', 'Heart Pain', 'Normal',
       'Heart Pain', 'Heart Pain', 'Normal', 'Heart Pain', 'Heart Pain',
       'Heart Pain', 'Heart Pain', 'Normal', 'Heart Pain', 'Heart Pain',
       'Heart Pain', 'Heart Pain', 'Normal', 'Normal', 'Heart Pain',
       'Heart Pain', 'Heart Pain', 'Normal', 'Normal', 'Heart Pain',
       'Normal', 'Heart Pain', 'Normal', 'Heart Pain', 'Normal',
       'Heart Pain', 'Normal', 'Heart Pain', 'Heart Pain', 'Normal',
       'Heart Pain', 'Normal', 'Heart Pain', 'Normal', 'Normal', 'Normal',
       'Heart Pain', 'Heart Pain', 'Heart Pain', 'Heart Pain', 'Normal',
       'Heart Pain', 'Normal', 'Heart Pain', 'Normal', 'Normal',
       'Heart Pain', 'Normal', 'Heart Pain', 'Heart Pain', 'Heart Pain',
       'Normal', 'Normal', 'Heart Pain', 'Heart Pain', 'Normal',
       'Heart Pain', 'Heart Pain', 'Heart Pain', 'Heart Pain', 'Normal',
       'Hear

In [47]:
y_test

471    Heart Pain
290    Heart Pain
537    Heart Pain
269    Heart Pain
124        Normal
          ...    
452    Heart Pain
454    Heart Pain
0          Normal
298    Heart Pain
508    Heart Pain
Name: class, Length: 165, dtype: object

In [48]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# Detection

In [49]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [50]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [52]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

AttributeError: module 'mediapipe.python.solutions.holistic' has no attribute 'FACEMESH_TELLESTATION'

In [ ]:
tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))